# Timing for QobjEvo

Made by Eric Giguere

In [1]:
import qutip as qt
import numpy as np
from qutip import QobjEvo
%load_ext cython

In [2]:
N = 10
destroy, create, Id = qt.destroy(N), qt.create(N), qt.qeye(N)
def exp_i(t,args):
    return np.exp(-1j*t)
def cos_w(t,args):
    return np.cos(args["w"]*t)
tlist = np.linspace(0,10,10000)
tlistlog = np.logspace(-3,1,10000)

# state vector as np array
vec = np.arange(N)*.5+.5j
vec_super = np.arange(N**2)*.5+.5j
mat_c = (np.arange(N**2)*.5+.5j).reshape((10,10))
mat_f = np.asfortranarray(mat_c*1.)

# Construct QobjEvo of all type
td_cte1 = QobjEvo(Id)
td_cte2 = QobjEvo([Id])

td_func = QobjEvo([Id,[create,exp_i],[destroy,cos_w]],args={"w":2})
td_str = QobjEvo([Id,[create,"exp(-1j*t)"],[destroy,"cos(w*t)"]],args={'w':2.})
td_array = QobjEvo([Id,[create,np.exp(-1j*tlist)],[destroy,np.cos(2*tlist)]],tlist=tlist)
td_array_log = QobjEvo([Id,[create,np.exp(-1j*tlistlog)],[destroy,np.cos(2*tlistlog)]],tlist=tlistlog)

td_super = qt.liouvillian(td_func, c_ops=td_cte1)

## Timing: call

Timing check for all coeff formats.  
Once compiled, the cython method is called by the usual method.

In [3]:
# The compiled version build the scipy object from the sparce matrix if not used with data=True
print("Time constant QobjEvo")
td_cte1.compiled = False
%timeit td_cte1(2)
%timeit td_cte1(2, data=True)
print("Time for compiled constant QobjEvo")
td_cte1.compile()
%timeit td_cte1(2)
%timeit td_cte1(2, data=True)

Time for cte
58.3 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
14.1 µs ± 82.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Time for compiled cte
55 µs ± 439 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
14.1 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
print("Time for str based coefficients")
td_str.compiled = False
%timeit td_str(2)
%timeit td_str(2, data=True)
td_str.compile()
%timeit td_str(2)
%timeit td_str(2, data=True)

Time for str
650 µs ± 8.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
227 µs ± 6.36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
74.2 µs ± 884 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
15.4 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
print("Time for function based coefficients")
td_func.compiled = False
%timeit td_func(2)
%timeit td_func(2, data=True)
td_func.compile()
%timeit td_func(2)
%timeit td_func(2, data=True)

Time for function
598 µs ± 5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
178 µs ± 2.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
83.3 µs ± 321 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
22.3 µs ± 273 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
print("Time for array based coefficients")
td_array.compiled = False
%timeit td_array(2)
%timeit td_array(2, data=True)
td_array.compile()
%timeit td_array(2)
%timeit td_array(2, data=True)

Time for array
653 µs ± 15.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
233 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
74.3 µs ± 438 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
15.4 µs ± 65.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [7]:
print("Time for array based coefficients with variable time step")
td_array_log.compiled = False
%timeit td_array_log(2)
%timeit td_array_log(2, data=True)
td_array_log.compile()
%timeit td_array_log(2)
%timeit td_array_log(2, data=True)

Time for array with variable time step
635 µs ± 2.62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
232 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
72.7 µs ± 247 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
15.2 µs ± 69.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing: multiplication with a vector
The QobjEvo at t x vec, used by most solver

In [8]:
print("timing for rhs: constant QobjEvo")
td_cte1.compiled = False
%timeit qt.cy.spmv(td_cte1(2, data=True), vec)
%timeit qt.cy.spmv(Id.data, vec)
%timeit td_cte1.mul_vec(2,vec)
td_cte1.compile()
%timeit td_cte1.mul_vec(2,vec)

timing for rhs: cte
24.5 µs ± 531 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.85 µs ± 63.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
29.9 µs ± 484 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
7.43 µs ± 87.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
print("timing for rhs: str based coefficients")
td_str.compiled = False
%timeit qt.cy.spmv(td_str(2, data=True), vec)
%timeit td_str.mul_vec(2,vec)
td_str.compile()
%timeit td_str.mul_vec(2,vec)

timing for rhs: str
238 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
246 µs ± 8.64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
9.88 µs ± 30.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
print("timing for rhs: function based coefficients")
td_func.compiled = False
%timeit qt.cy.spmv(td_func(2, data=True), vec)
%timeit td_func.mul_vec(2,vec)
td_func.compile()
%timeit td_func.mul_vec(2,vec)

timing for rhs: cte
188 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
194 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
16.7 µs ± 79.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
print("timing for rhs: array based coefficients")
td_array.compiled = False
%timeit qt.cy.spmv(td_array(2, data=True), vec)
%timeit td_array.mul_vec(2,vec)
td_array.compile()
%timeit td_array.mul_vec(2,vec)

timing for rhs: cte
238 µs ± 7.72 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
243 µs ± 7.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
9.96 µs ± 65.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
print("timing for rhs: logscaled array based coefficients")
td_array_log.compiled = False
%timeit qt.cy.spmv(td_array_log(2, data=True), vec)
%timeit td_array_log.mul_vec(2,vec)
td_array_log.compile()
%timeit td_array_log.mul_vec(2,vec)

timing for rhs: cte
236 µs ± 3.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
256 µs ± 7.73 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
10.1 µs ± 59.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing: multiplication with matrix
QobjEvo at t time same shape matrix, used for operator evolution

In [13]:
print("timing for mul_mat: c format matrice, sparce QobjEvo")
td_str.compiled = False
%timeit td_str(2, data=True) * mat_c
%timeit td_str.mul_mat(2,mat_c)
td_str.compile()
%timeit td_str.mul_mat(2,mat_c)

timing for mul_mat: c format sparce
242 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
252 µs ± 2.72 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
11.9 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
print("timing for mul_mat: c format matrice, dense QobjEvo")
td_str.compiled = False
%timeit td_str(2, data=True) * mat_c
%timeit td_str.mul_mat(2,mat_c)
td_str.compile(dense=True)
%timeit td_str.mul_mat(2,mat_c)

timing for mul_mat: c format dense
239 µs ± 1.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
254 µs ± 9.27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
16.9 µs ± 579 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [15]:
print("timing for mul_mat: f format matrice, sparce QobjEvo")
td_str.compiled = False
%timeit td_str(2, data=True) * mat_f
%timeit td_str.mul_mat(2,mat_f)
td_str.compile()
%timeit td_str.mul_mat(2,mat_f)

timing for mul_mat: f format sparce
241 µs ± 3.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
260 µs ± 7.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
13.7 µs ± 177 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [16]:
print("timing for mul_mat: f format matrice, dense QobjEvo")
td_str.compiled = False
%timeit td_str(2, data=True) * mat_f
%timeit td_str.mul_mat(2,mat_f)
td_str.compile(dense=True)
%timeit td_str.mul_mat(2,mat_f)

timing for mul_mat: f format dense
242 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
253 µs ± 4.64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
15.9 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing: expect
expectation value of QobjEvo(t) and psi/rho

In [17]:
print("timing for expect: cte QobjEvo")
td_cte1.compiled = False
%timeit qt.cy.cy_expect_psi(td_cte1(2, data=True), vec, 0)
%timeit td_cte1.expect(2,vec,0)
td_cte1.compile()
%timeit td_cte1.expect(2,vec,0)

timing for expect: cte
20.2 µs ± 333 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
29.8 µs ± 527 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
9.81 µs ± 38.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [18]:
print("timing for expect: str based coefficient")
td_str.compiled = False
%timeit qt.cy.cy_expect_psi(td_str(2, data=True), vec, 0)
%timeit td_str.expect(2,vec,0)
td_str.compile()
%timeit td_str.expect(2,vec,0)

timing for expect: str
234 µs ± 4.36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
248 µs ± 7.23 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
12.7 µs ± 26.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [19]:
print("timing for expect: func based coefficient")
td_func.compiled = False
%timeit qt.cy.cy_expect_psi(td_func(2, data=True), vec, 0)
%timeit td_func.expect(2,vec,0)
td_func.compile()
%timeit td_func.expect(2,vec,0)

timing for expect: func
185 µs ± 900 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
194 µs ± 945 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
19.5 µs ± 62.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
print("timing for expect: array based coefficient")
td_array.compiled = False
%timeit qt.cy.cy_expect_psi(td_array(2, data=True), vec, 0)
%timeit td_array.expect(2,vec,0)
td_array.compile()
%timeit td_array.expect(2,vec,0)

timing for expect: array
236 µs ± 7.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
244 µs ± 8.32 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
12.5 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
print("timing for expect: array logscale based coefficient")
td_array_log.compiled = False
%timeit qt.cy.cy_expect_psi(td_array_log(2, data=True), vec, 0)
%timeit td_array_log.expect(2,vec,0)
td_array_log.compile()
%timeit td_array_log.expect(2,vec,0)

timing for expect: array logscale
233 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
248 µs ± 9.68 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
12.6 µs ± 58.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [22]:
print("timing for expect_rho: func based coefficient")
td_super.compiled = False
%timeit qt.cy.cy_expect_rho_vec(td_super(2, data=True), vec_super, 0)
%timeit td_super.expect(2,vec_super,0)
td_super.compile()
%timeit td_super.expect(2,vec_super,0)

timing for expect_rho: func
195 µs ± 2.02 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
205 µs ± 4.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
16 µs ± 40.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing: call with other arguments

In [23]:
td_args = QobjEvo([Id,[destroy,cos_w]],args={"w":1})

In [24]:
print("timing for function based coefficient")
%timeit td_args(2, args={"w":2})
%timeit td_args(2)
%timeit td_args(2, args={"w":2}, data=True)
%timeit td_args(2, data=True)
td_args.compile()
%timeit td_args(2, args={"w":2})
%timeit td_args(2)
%timeit td_args(2, args={"w":2}, data=True)
%timeit td_args(2, data=True)

timing for function
330 µs ± 5.66 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
322 µs ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
99.6 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
95.6 µs ± 257 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
328 µs ± 4.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
78.2 µs ± 536 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
99.8 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
18 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [25]:
td_args = QobjEvo([Id,[destroy,"cos(w*t)"]],args={"w":1})

In [26]:
print("timing for string based coefficient")
%timeit td_args(2, args={"w":2})
%timeit td_args(2)
%timeit td_args(2, args={"w":2}, data=True)
%timeit td_args(2, data=True)
td_args.compile()
%timeit td_args(2, args={"w":2})
%timeit td_args(2)
%timeit td_args(2, args={"w":2}, data=True)
%timeit td_args(2, data=True)

timing for string
359 µs ± 5.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
350 µs ± 3.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
123 µs ± 430 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
121 µs ± 789 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
353 µs ± 3.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
72.5 µs ± 212 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
126 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
14.7 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing: call that depend on state


In [27]:
def coeff_state(t, args):
    return np.max(args["vec"])*args["w"]
td_state = QobjEvo([Id, [destroy, coeff_state]],args={"w":1,"vec=vec":vec})
%timeit td_state(2, state=vec, data=True)
td_state.compile()
%timeit td_state(2, state=vec, data=True)

106 µs ± 557 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
32.7 µs ± 337 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [28]:
def coeff_state(t, args):
    return args["e1"]
td_state = QobjEvo([Id, [destroy, coeff_state]], args={"e1=expect":qt.qeye(N)})
%timeit td_state(2, state=vec, data=True)
td_state.compile()
%timeit td_state(2, state=vec, data=True)

128 µs ± 688 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
30.7 µs ± 106 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [29]:
td_state = QobjEvo([Id, [destroy, "e1"]], args={"e1=expect":qt.qeye(N)})
%timeit td_state(2, state=vec, data=True)
td_state.compile()
%timeit td_state(2, state=vec, data=True)

148 µs ± 629 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
30.2 µs ± 741 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [30]:
qt.about()


QuTiP: Quantum Toolbox in Python
Copyright (c) 2011 and later.
A. J. Pitchford, P. D. Nation, R. J. Johansson, A. Grimsmo, and C. Granade

QuTiP Version:      4.4.0.dev0+42af593a
Numpy Version:      1.16.2
Scipy Version:      1.2.1
Cython Version:     0.29.6
Matplotlib Version: 3.0.3
Python Version:     3.7.3
Number of CPUs:     3
BLAS Info:          INTEL MKL
OPENMP Installed:   False
INTEL MKL Ext:      True
Platform Info:      Linux (x86_64)
Installation path:  /home/eric/anaconda3/lib/python3.7/site-packages/qutip-4.4.0.dev0+42af593a-py3.7-linux-x86_64.egg/qutip
Please cite QuTiP in your publication.
For your convenience a bibtex reference can be easily generated using `qutip.cite()`
